<div class="alert alert-danger">

<h1>Take notice!</h1>
<ul>
    <li>This class will be recorded</li>
    <li>This lab uses a lot of memory. Shut down all other notebooks!</li>
</ul>
    
</div>

# Spatial Autocorrelation

<img src="images/sa.png" width=800>

Visual interpretations are meaningful ways to determine spatial trends in our data. However, underlying factors—such as inconsistent geographies, scale, data gaps, overlapping data—have the potential to produce incorrect assumptions, as valuable information may be conveniently hidden from the visual output.

<a href="https://en.wikipedia.org/wiki/Spatial_analysis#/media/File:Snow-cholera-map.jpg" target="_blank"><img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/Snow-cholera-map.jpg" width=400></a>

One way to address this issue is to amend your visual output with geo-statistical validation. In this lab, we will look at one such approach: Spatial Autocorrelation. Spatial autocorrelation addresses the so-called "First Law of Geography":

> “Everything is related to everything else. But near things are more related than distant things”. Waldo Tobler’s (1969) First Law of Geography

In other words, things that happen somewhere are likely to also happen at nearby locations.

## Methodology
In this lab, we take on the controversial topic of policing in Los Angeles, specifically looking at arrest records from the LAPD. Do arrest locations have a statistical significant tendency to cluster in certain communities? To answer this question, we not only look at the location of recorded arrests in the city, but compare these locations with other arrests. In short, we are seeking to see where spatial correlations occur based on the data. Our approach is:

1. import census block group boundaries for Los Angeles
1. import arrest data from the LA Open Data Portal
1. spatially join the two datasets
1. normalize the data to create arrests per 1000
1. conduct [global spatial autocorrelation](https://geographicdata.science/book/notebooks/06_spatial_autocorrelation.html) using Moran's I
1. conduct [local spatial autocorrelation](https://geographicdata.science/book/notebooks/07_local_autocorrelation.html) using Local Indicators of Spatial Association (LISAs)

## Libraries to use

<img src="images/esda.png">

<img src="images/splot.png">

- [ESDA](https://pysal.org/esda/)
- [SPLOT](https://github.com/pysal/splot)

In [ ]:
# to read and wrangle data
import pandas as pd

# to import data from LA Data portal
from sodapy import Socrata

# to create spatial data
import geopandas as gpd

# for basemaps
import contextily as ctx

# For spatial statistics
import esda
from esda.moran import Moran, Moran_Local

import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster,plot_moran_simulation

import libpysal as lps

# Graphics
import matplotlib.pyplot as plt
import plotly.express as px

## Block Groups

Our first task is to bring in a geography that will allow us to summarize the location of arrests. The smaller the geography, the better our spatial correlation results will be. Short of creating our own grid, the census block groups provides an easily accessible boundary layer at a human scale. Additionally, working with census geographies will allow for future analyses that may include census data.

* Data source: 
   * [Census Reporter: ACS 2021 5 year: Table B01003: Total Population in Los Angeles: Census Block Groups](https://censusreporter.org/data/table/?table=B01003&geo_ids=16000US0644000,150|16000US0644000&primary_geo_id=16000US0644000)

## Why block groups?

Allow me to digress. The choice to use census tracts vs census block groups is an important one.

-[Census Blocks and Block Groups](https://www2.census.gov/geo/pdfs/reference/GARM/Ch11GARM.pdf)

In [ ]:
# to illustrate, bring in tracts
gdf_tracts = gpd.read_file('data/tracts.geojson')

In [ ]:
# population block group data from census reporter
# gdf_bg = gpd.read_file('data/acs2019_5yr_B01003_15000US060372672002.geojson')
gdf_bg = gpd.read_file('data/acs2021_5yr_B01003_15000US060371041082.geojson')

In [ ]:
# compare
fig,ax = plt.subplots(figsize=(15,15))

# blue background of census tracts
gdf_tracts.plot(ax=ax,zorder=10,alpha=0.5)

# while lines to show block groups (sandwiched between the two layers above for visual clarity)
gdf_bg.boundary.plot(ax=ax,color='white',zorder=15,lw=0.5)

# black boundary lines of census tracts on top of everything
gdf_tracts.boundary.plot(ax=ax,color='black',zorder=20,lw=1)

Let's zoom in...

In [ ]:
# compare
fig,ax = plt.subplots(figsize=(20,15))

# blue background of census tracts
gdf_tracts.plot(ax=ax,zorder=10,alpha=0.5)

# black boundary of census tracts
gdf_tracts.boundary.plot(ax=ax,color='black',zorder=20,lw=2)

# while lines to show block groups (sandwiched between the two layers above for visual clarity)
gdf_bg.boundary.plot(ax=ax,color='white',zorder=15,lw=0.8,linestyle='--')

# zoom in

# get the total bounds
minx,miny,maxx,maxy = gdf_bg.total_bounds

# set the x and y limits manually
ax.set_xlim(minx+0.2,maxx-0.2) # zooming in just a bit
ax.set_ylim(miny+0.3,maxy-0.3)

## Data cleanup

In [ ]:
# what does our data look like?
gdf_bg.info()

In [ ]:
# trim the data to the bare minimum columns
gdf_bg = gdf_bg[['geoid','B01003001','geometry']]

# rename the columns
gdf_bg.columns = ['FIPS','TotalPop','geometry']

In [ ]:
# last rows
gdf_bg.tail()

<div class="alert alert-danger">

<h1>Take notice!</h1>

Data downloaded from census reporter includes a **summary** row, meaning that a single row will represent the total of all other rows.

In this case, the last row is a summary of the total population for all of LA. To conduct a correct spatial analysis, **make sure to delete this row** before proceeding.
    
</div>

In [ ]:
# delete last row which is for the entire city of LA
gdf_bg.drop(gdf_bg.tail(1).index,inplace=True)

In [ ]:
# fix FIPS code
gdf_bg['FIPS'] = gdf_bg['FIPS'].str.replace('15000US','')
gdf_bg.tail()

One more data cleanup: get rid of census blocks groups with less than 100 total population.

In [ ]:
# sort by total pop
gdf_bg.sort_values(by='TotalPop').head(20)

In [ ]:
# delete less than 100 population geographies
gdf_bg = gdf_bg[gdf_bg['TotalPop']>100]

## Map the census block groups

In [ ]:
# get the layers into a web mercator projection
# reproject to web mercator
gdf_bg = gdf_bg.to_crs(epsg=3857)

In [ ]:
# plot it!
fig, ax = plt.subplots(figsize=(12,12))

gdf_bg.plot(ax=ax,
         color='black', 
         edgecolor='white',
         lw=0.5,
         alpha=0.4)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

## Get Arrest Data from LA Open Data Portal
Next, we acquire the data using the socrata API. Use the socrata documentation to grab the code syntax for our arrests data.
- https://dev.socrata.com/foundry/data.lacity.org/amvf-fr72

In [ ]:
# connect to the data portal
client = Socrata("data.lacity.org", None)

results = client.get("amvf-fr72", 
                     limit=50000,
                     where = "arst_date between '2022-07-01T00:00:00' and '2023-2-28T00:00:00'",
                     order='arst_date desc')

# Convert to pandas DataFrame
arrests = pd.DataFrame.from_records(results)


In [ ]:
# if socrata is slow or does not work, uncomment the line below to load a backup version
# arrests = pd.read_csv('data/arrests.csv')

In [ ]:
arrests.shape

### Convert data to a geodataframe

Geopandas allows us to convert different types of data into a spatial format.
- https://geopandas.org/gallery/create_geopandas_from_pandas.html

In [ ]:
# convert pandas dataframe to geodataframe
arrests = gpd.GeoDataFrame(arrests, 
                         crs='EPSG:4326',
                         geometry=gpd.points_from_xy(arrests.lon, arrests.lat))

In [ ]:
# get the layers into a web mercator projection
# reproject to web mercator
arrests = arrests.to_crs(epsg=3857)

In [ ]:
# convert lat/lon to floats
arrests.lon = arrests.lon.astype('float')
arrests.lat = arrests.lat.astype('float')

In [ ]:
# map it!
fig,ax = plt.subplots(figsize=(12,12))

arrests.plot(ax=ax,
             color='red',
             markersize=1)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)


### The 0,0 conundrum
What is that red dot off the coast of Africa? Yes, that is the infamous spatial black hole, the [0,0] coordinate. There can be many reasons why those red dots get lost and find themselves there. No data may default to 0's, null values may be converted to 0's, or a human may have inadvertently entered 0's for unknown locations. Either which way, since these records do not have valid locations, they need to be deleted from our dataframe in order to proceed.

In [ ]:
# subset the zero coordinate records
arrests[arrests.lon==0]

In [ ]:
# drop the unmapped rows
arrests = arrests[arrests.lon!=0]

In [ ]:
# another way to do this
# arrests.drop(arrests[arrests.lon==0].index,inplace=True)

In [ ]:
# map it!
fig,ax = plt.subplots(figsize=(20,20))

arrests.plot(ax=ax,
             color='red',
             markersize=10, alpha=0.1)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)


## Create a two layer map

- https://geopandas.org/mapping.html

Since we want to zoom to the extent of the arrests layer (and not the block groups), get the bounding coordinates for our axis.

In [ ]:
# get the bounding box coordinates for the arrest data
minx, miny, maxx, maxy = arrests.geometry.total_bounds
print(minx)
print(maxx)
print(miny)
print(maxy)

## Subplots for multi-layered maps

For our multi-layered maps, we are taking it one step further from our previous lab using matplotlib's `subplots`. `subplots` allows the creation of multiple plots on a gridded canvas. For our map, we only need a single subplot, but we are layering multiple datasets *on top of one another* on that subplot. To specify which subplot to put the layer on, you use the `ax` argument.

In [ ]:
# set up the plot canvas with plt.subplots
fig, ax = plt.subplots(figsize=(20, 20))

# block groups
gdf_bg.plot(ax=ax, # this puts it in the ax plot
        color='gray', 
        edgecolor='white',
        alpha=0.4)

# arrests
arrests.plot(ax=ax, # this also puts it in the same ax plot
        color='red',
        markersize=10,
        alpha=0.2)

# use the bounding box coordinates to set the x and y limits
ax.set_xlim(minx - 1000, maxx + 1000) # added/substracted value is to give some margin around total bounds
ax.set_ylim(miny - 1000, maxy + 1000)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

## The spatial join

How many arrests are there in each census block group? In order to answer this question, one must *count* the number of arrests that fall within each block group boundary. To do so, we conduct a **spatial join**.

* https://geopandas.org/mergingdata.html?highlight=spatial%20join

In a Spatial Join, two geometry objects are merged based on their spatial relationship to one another.

While the official documentation may seem confusing, consider the following as a rule of thumb. When you do a spatial join with `gpd.sjoin()`, you feed it three arguments: a left dataframe, a right dataframe, and a how statement.

- **Left dataframe**: identify the layer you want *to* attach infomation that will come from the other layer
- **Right dataframe**: identify the layer that you want to get information *from* to attach to the other layer

Once you identify your left and right dataframes, use `how="left"` to spatially join the two layers (think: "I'm sending data from the right to the left"). Note that this will result in a dataframe with the same records and datatype as the LEFT layer if it is a perfect one to one match. However, if there are multiple matches from the right dataframe to the left, **it will create multiple rows**, one for each match.

In [ ]:
# check number of records before join
arrests.shape

In [ ]:
# Do the spatial join
join = gpd.sjoin(gdf_bg, arrests, how='left')

In [ ]:
# check number of records after join
join.shape

This creates a dataframe that has every arrest record with the corresponding FIPS code.

Next, we create another dataframe that counts crime by their corresponding block group:

In [ ]:
arrests_by_bg = join.FIPS.value_counts().rename_axis('FIPS').reset_index(name='arrests_count')
arrests_by_bg

In [ ]:
# make a bar chart of top 20 geographies
arrests_by_bg[:20].plot.bar(figsize=(20,4),
                             x='FIPS',
                             y='arrests_count')

Nice chart... but is it meaningful?

In [ ]:
arrests_by_bg.plot.hist(bins=100)

In [ ]:
plt.boxplot(arrests_by_bg.arrests_count)

## Join the value counts back to the gdf

How many people know their census block number? The bar chart is nice, but it is not informative. Without spatial awareness, the data chart does little to convey knowledge. What we want is a choropleth map to accompany it. To do so, we merge the counts back to the block group gdf.

In [ ]:
# join the summary table back to the gdf
gdf_bg=gdf_bg.merge(arrests_by_bg,on='FIPS')

Now the block group gdf has a new column for arrest counts:

In [ ]:
# our block group table now has a count column
gdf_bg.sample(10)

## Normalizing: Arrests per 1000 people
Rather than proceeding with an absolute count of arrests, let's normalize it by number of people who live in the census block group.



In [ ]:
gdf_bg['arrests_per_1000'] = gdf_bg['arrests_count']/gdf_bg['TotalPop']*1000

In [ ]:
gdf_bg.sort_values(by="arrests_per_1000").tail()

And if you are curious, we can map a slice of the data. Here, we sort the values by descending arrest rate, and only show a slice of the data, the top 20 geographies using the handy `[:20]`.

In [ ]:
# map the top 20 geographies
fig,ax = plt.subplots(figsize=(12,10))
gdf_bg.sort_values(by='arrests_per_1000',ascending=False)[:10].plot(ax=ax,
                                                                 color='red',
                                                                 edgecolor='white',
                                                                 alpha=0.5)


# title
ax.set_title('Top 20 locations of LAPD arrests per 1000 people (July 2022-January 2023)',fontsize=15,pad=20)

# no axis
ax.axis('off')

# add a basemap
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

## Choropleth map of arrests

Finally, we are ready to generate a choropleth map of arrests. 

In [ ]:
fig,ax = plt.subplots(figsize=(15,15))

gdf_bg.plot(ax=ax,
        column='arrests_per_1000', # this defines the field to "choropleth"
        legend=True,
        alpha=0.8,
        cmap='RdYlGn_r', # the "_r" reverses the color
        scheme='quantiles')

ax.axis('off')
ax.set_title('2022 July to January 2023 arrests per 1000 people',fontsize=18,pad=20)
ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

<div class="alert alert-info">
The map above is a good way to begin exploring spatial patterns in our data. What does this map tell you? Is it informative? Do you notice any significant clusters? What if you change the map? Notice the `scheme` argument is set to `naturalbreaks`. Experiment with other map classfications such as `equalinterval`, `quantiles`. How does each classification change the map? 
</div>

In [ ]:
# a quick interactive version: geopandas .explore function does it all, no need to import folium!
gdf_bg.explore(column='arrests_per_1000', # this defines the field to "choropleth"
        legend=True,
        cmap='RdYlGn_r', # the "_r" reverses the color
        scheme='naturalbreaks',
        tiles='CartoDB positron',
        style_kwds={
            'weight':0.5,
            'color':'black',
            'opacity':0.5
        })

# Global Spatial Autocorrelation
We have imported two datasets. Cleaned them up, spatialized them, and connected them spatially. We successfully mapped them to show the location of arrests per 1000 people by census block groups. The resulting map intuitively and visually tells us that there does appear to be spatial clusters of where arrests are more prevalent, but to what degree of certainty can we say so? Actually, very little, without statitistically backing up our determinations. Could this exact pattern be a matter of chance? Or is the pattern so distinct that there is no way it could have happened randomly?

In order to answer this question, we conduct spatial autocorrelation, a process that determines to what degree an existing pattern is or is not random.

Global Moran's I statistic is a way to *quantify* the degree to which similar geographies are clustered. To do so, we compare each geography based on a given value (in this case arrest counts) with that of its neighbors. The first step of this process is to define a "spatial weight." 

### Spatial Weights

<img src="https://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-statistics-toolbox/GUID-A772FE33-6EE7-4B45-A654-A3C73235E9A2-web.gif" width=400>

Image source: [ESRI](https://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-statistics-toolbox/generate-spatial-weights-matrix.htm)

Spatial weights are how we determine the area’s neighborhood. There are different statistical methods that are used for determining spatial weights. Some use a contiguity model, assigning neighbors based on boundaries that touch each other. Others are based on distance, finding the closest neighbors based on the centroid of each geography. So which method should we use? 

For this lab, we will use the KNN weight, where `k` is the number of "nearest neighbors" to count in the calculations. Let's proceed with `k=8` for our KNN spatial weights. 

We also **row standardize** the data: A technique for adjusting the weights in a spatial weights matrix. When weights are row standardized, each weight is divided by its row sum. The row sum is the sum of weights for a features neighbors.

- https://geographicdata.science/book/notebooks/04_spatial_weights.html#distance-based-weights

In [ ]:
# calculate spatial weight
wq =  lps.weights.KNN.from_dataframe(gdf_bg,k=8)

# Row-standardization
wq.transform = 'r'

### Spatial lag

Now that we have our spatial weights assigned, we use it to calculate the spatial lag. While the mathematical operations are beyond the scope of this lab, you are welcome to check it out [here](https://geographicdata.science/book/notebooks/06_spatial_autocorrelation.html#spatial-lag). Simply put, the spatial lag is a calculated assignment to each geography in your data, which takes into account the data values from others in their  "neighborhood" as defined by the spatial weight. This operation can be done with a single line of code which is part of the pysal module, but the underlying calculations are not that difficult to understand: 

It takes the average of all the neighbors as defined by the spatial weight to come up with a single associated value.

In [ ]:
# create a new column for the spatial lag
gdf_bg['arrests_per_1000_lag'] = lps.weights.lag_spatial(wq, gdf_bg['arrests_per_1000'])

In [ ]:
# take a look at some sample rows
gdf_bg.sample(10)[['FIPS','TotalPop','arrests_count','arrests_per_1000','arrests_per_1000_lag']]

<div class="alert alert-info">
Take a moment to look at the values in the dataframe. What do they tell you?
</div>

### The donut and the diamond

In [ ]:
# create a column that calculates the difference between arrests and lag
gdf_bg['arrest_lag_diff'] = gdf_bg['arrests_per_1000'] - gdf_bg['arrests_per_1000_lag']

In [ ]:
# output to get the head and tail
gdf_bg.sort_values(by='arrest_lag_diff')

In order to better understand the significance of the spatial lag values, consider the following two geographies:

In [ ]:
# the FIPS with highest negative difference
gdf_donut = gdf_bg.sort_values(by='arrest_lag_diff').head(1)
gdf_donut

In [ ]:
# the FIPS with highest positive difference
gdf_diamond = gdf_bg.sort_values(by='arrest_lag_diff').tail(1)
gdf_diamond

### Satellite exploration

To better illustrate our assumptions, let's display these locations using satellite imagery. 

Geopandas has a new function that allows for quick folium interactive maps as seen in the code above. You can find the documentation here:

https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html

Furthermore, we can use a free satellite basemap from ESRI. You can preview it and many others from this URL:

https://leaflet-extras.github.io/leaflet-providers/preview/

In [ ]:
# map the donut
gdf_donut.explore(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                  attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community')

In [ ]:
# map the diamond
gdf_diamond.explore(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
                  attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community')

What's the story here?

## Spatial lag map
But we digress. Let's map the entire dataframe by the newly created spatial lag column.

In [ ]:
# use subplots that make it easier to create multiple layered maps
fig, ax = plt.subplots(figsize=(15, 15))

# spatial lag choropleth
gdf_bg.plot(ax=ax,
         figsize=(15,15),
         column='arrests_per_1000_lag',
         legend=True,
         alpha=0.8,
         cmap='RdYlGn_r',
         scheme='quantiles')

ax.axis('off')
ax.set_title('July 2022 - January 2023 Arrests per 1000 people',fontsize=20,pad=20)

ctx.add_basemap(ax,source=ctx.providers.CartoDB.Positron)

## Side-by-side maps
We can now compare these two map outputs side by side. Notice that the syntax is a bit different from past labs where we have only worked with one figure at a time. This output produces 1 row, and 2 columns of figures in `subplots`.
- [subplots documentation](https://matplotlib.org/3.3.0/gallery/subplots_axes_and_figures/subplots_demo.html)

In [ ]:
# create the 1x2 subplots
fig, ax = plt.subplots(1, 2, figsize=(20, 12))

# two subplots produces ax[0] (left) and ax[1] (right)

# regular count map on the left
gdf_bg.plot(ax=ax[0], # this assigns the map to the left subplot
         column='arrests_per_1000', 
         cmap='RdYlGn_r', 
         scheme='quantiles',
         k=5, 
         edgecolor='white', 
         linewidth=0, 
         alpha=0.75, 
           )


ax[0].axis("off")
ax[0].set_title("Arrests per 1000")

# spatial lag map on the right
gdf_bg.plot(ax=ax[1], # this assigns the map to the right subplot
         column='arrests_per_1000_lag', 
         cmap='RdYlGn_r', 
         scheme='quantiles',
         k=5, 
         edgecolor='white', 
         linewidth=0, 
         alpha=0.75
           )

ax[1].axis("off")
ax[1].set_title("Arrests per 1000 - Spatial Lag")

plt.show()

## Interactive spatial lag satellite map
Building the equivalent map as an interactive javascript map is a bit more challenging. While there are several options to choose from, this lab will use plotly express's choropleth_mapbox feature.
- https://plotly.com/python/mapbox-county-choropleth/#

In [ ]:
# a quick interactive version: geopandas .explore function does it all, no need to import folium!
gdf_bg.explore(
        column='arrests_per_1000_lag', # this defines the field to "choropleth"
        legend=True,
        cmap='RdYlGn_r', # the "_r" reverses the color
        scheme='naturalbreaks',
        style_kwds={
            'weight':0.5,
            'color':'black',
            'opacity':0.5
        },
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
        )

## Moran's Plot

We now have a spatial lag map: a map that displays geographies weighted against the values of its neighbors. The clusters are much clearer and cleaner than the original arrest count map. Downtown, Venice, South LA, Van Nuys... But we still have not *quantified* the degree of the spatial correlations. To begin this process, we test for global autocorrelation for a continuous attribute (arrest counts).

In [ ]:
y = gdf_bg.arrests_per_1000
moran = Moran(y, wq)
moran.I

The moran's I value is nothing more than the calculated slope of the scatterplot of our "arrests per 1000" and "arrests per 1000 spatial lag" columns. It does indicate whether or not you have a positive or negative autocorrelation. Values will range from positive one, to negative one. 

- **Positive** spatial autocorrelation: high values are close to high values, and/or low values are close to low values
- **Negative** spatial autocorrelation (less common): similar values are far from each other; high values are next to low values, low values are next to high values

You can output a scatterplot:

In [ ]:
fig, ax = moran_scatterplot(moran, aspect_equal=True)
plt.show()

So what is the significance of our Moran value? In other words, **how likely is our observed pattern on the map generated by an entirely random process?** To find out, we compare our value with a simulation of 999 permutations that randomly shuffles the arrest data throughout the given geographies. The output is a sampling distribution of Moran’s I values under the (null) hypothesis that attribute values are randomly distributed across the study area. We then compare our observed Moran’s I value to this "Reference Distribution."

In [ ]:
plot_moran_simulation(moran,aspect_equal=False)

We can compute the P-value:

<img src="https://www.simplypsychology.org/p-value.png" width=600>

In [ ]:
moran.p_sim

The value is calculated as an empirical P-value that represents the proportion of realisations in the simulation under spatial randomness that are more extreme than the observed value. A small enough p-value associated with the Moran’s I of a map allows to reject the hypothesis that the map is random. In other words, we can conclude that the map displays more spatial pattern than we would expect if the values had been randomly allocated to a locations.

That is a very low value, particularly considering it is actually the minimum value we could have obtained given the simulation behind it used 999 permutations (default in PySAL) and, by standard terms, it would be deemed statistically significant. We can ellaborate a bit further on the intuition behind the value of p_sim. If we generated a large number of maps with the same values but randomly allocated over space, and calculated the Moran’s I statistic for each of those maps, only 0.1% of them would display a larger (absolute) value than the one we obtain from the observed data, and the other 99.9% of the random maps would receive a smaller (absolute) value of Moran’s I. 

# Local Spatial Autocorrelation
So far, we have only determined that there is a positive spatial autocorrelation between the number of arrests in census block groups and their locations. But we have not detected where clusters are. Local Indicators of Spatial Association (LISA) is used to do that. LISA classifies areas into four groups: high values near to high values (HH), Low values with nearby low values (LL), Low values with high values in its neighborhood, and vice-versa.

- HH: high arrest rate geographies near other high arrest rate neighbors
- LL: low arrest rate geographies near other low arrest rate neighbors
- LH (donuts): low arrest rate geographies surrounded by high arrest neighbors
- HL (diamonds): high arrest rate geographies surrounded by low arrest neighbors

## Moran Local Scatterplot

In [ ]:
# calculate local moran values
lisa = esda.moran.Moran_Local(y, wq)

In [ ]:
# Plot
fig,ax = plt.subplots(figsize=(20,15))

moran_scatterplot(lisa, ax=ax, p=0.05)
ax.set_xlabel("Arrests")
ax.set_ylabel('Spatial Lag of Arrests')

# add some labels
plt.text(1.95, 0.5, "HH", fontsize=25)
plt.text(1.95, -1, "HL", fontsize=25)
plt.text(-2, 1, "LH", fontsize=25)
plt.text(-1, -1, "LL", fontsize=25)
plt.show()

In the scatterplot above, the colored dots represents the rows (census block groups) that have a P-value less that 0.05 in each quadrant. In other words, these are the statisticaly significantly, spatially autocorrelated geographies.

## Spatial Autocorrelation Map
Finally, you can visually these statistically significant clusters using the `lisa_cluster` function:

In [ ]:
fig, ax = plt.subplots(figsize=(14,12))
lisa_cluster(lisa, gdf_bg, p=0.05, ax=ax)
plt.show()

And create a map comparing different p-values

In [ ]:
# create the 1x2 subplots
fig, ax = plt.subplots(1, 2, figsize=(20, 12))

# regular count map on the left
lisa_cluster(lisa, gdf_bg, p=0.05, ax=ax[0])

ax[0].axis("off")
ax[0].set_title("P-value: 0.05")

# spatial lag map on the right
lisa_cluster(lisa, gdf_bg, p=0.01, ax=ax[1])
ax[1].axis("off")
ax[1].set_title("P-value: 0.01")

plt.show()

Discuss: What are some conclusions you can come up with from these outputs?

# Resources

- https://geographicdata.science/book/notebooks/06_spatial_autocorrelation.html
- https://pysal.org/esda/notebooks/spatialautocorrelation.html
- https://towardsdatascience.com/what-is-exploratory-spatial-data-analysis-esda-335da79026ee